In [1]:
%pip install datasets evaluate
%pip install transformers[torch]
%pip install accelerate -U

# Please restart the runtime after installing packages

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. Data preparation

In [3]:
from datasets import load_dataset

imdb = load_dataset("imdb")

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_imdb = imdb.map(preprocess_function, batched=True, num_proc=10)

Map (num_proc=10):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Model

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# Training

In [8]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.282000,0.216251,0.915120
2,0.149400,0.200550,0.929160
3,0.102600,0.250929,0.924840
4,0.055700,0.321075,0.930280
5,0.038100,0.344045,0.928200
6,0.021000,0.375231,0.928480
7,0.020200,0.384939,0.928680
8,0.010400,0.413715,0.930320
9,0.009400,0.428601,0.930840
10,0.005800,0.441084,0.930840


TrainOutput(global_step=7820, training_loss=0.06556177182728068, metrics={'train_runtime': 9618.1888, 'train_samples_per_second': 25.992, 'train_steps_per_second': 0.813, 'total_flos': 3.310144411949693e+16, 'train_loss': 0.06556177182728068, 'epoch': 10.0})

In [10]:
trainer.evaluate(tokenized_imdb["test"])

{'eval_loss': 0.20054994523525238,
 'eval_accuracy': 0.92916,
 'eval_runtime': 253.348,
 'eval_samples_per_second': 98.678,
 'eval_steps_per_second': 3.087,
 'epoch': 10.0}